In [1]:
import matplotlib.pyplot as plt
import matplotlib.animation
%matplotlib ipympl
import numpy as np
import pandas as pd

from scipy.spatial import KDTree, cKDTree
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.optimize import curve_fit

import time
import joblib
from tqdm import tqdm
import trackpy as tp
from numba import njit, prange

from yupi import Trajectory
import yupi.graphics as yg
import yupi.stats as ys

from utility import get_imsd, get_imsd_windowed, get_emsd, get_emsd_windowed, fit_hist, MB_2D,\
                    normal_distr, lorentian_distr, get_trajs, speed_windowed, theta_windowed, \
                    get_smooth_trajs, get_velocities
                   
show_verb = True
save_verb = True
anim_show_verb = False

In [2]:
rawTrajs = pd.read_parquet("../tracking/results/parquet/pre_merge_tracking.parquet")
red_particle_idx = 17
rawTrajs.loc[rawTrajs.particle != red_particle_idx, ["color"]] = "#00007F"
rawTrajs.loc[rawTrajs.particle == red_particle_idx, ["color"]] = "#FF0000"
colors = rawTrajs.loc[rawTrajs.frame == 0, 'color'].values
nDrops = len(rawTrajs.loc[rawTrajs.frame==0])
nFrames = max(rawTrajs.frame) + 1
print(f"nDrops:{nDrops}")
print(f"nFrames:{nFrames} --> {nFrames/10:.2f} s")


# WINDOWED ANALYSIS PARAMETERS
window = 3200 # 320 s
stride = 100 # 10 s
print(f"window of {window/10} s, stride of {stride/10} s")
startFrames = np.arange(0, nFrames-window, stride, dtype=int)
endFrames = startFrames + window
nSteps = len(startFrames)
print(f"number of steps: {nSteps}")


# step 10 with a 10 fps video --> 1 s
units = "px/s"
default_kwargs_blue = {"color": "#00FFFF", "ec": (0, 0, 0, 0.6), "density": True}
default_kwargs_red = {"color": "#EE4B2B", "ec": (0, 0, 0, 0.6), "density": True}

nDrops:50
nFrames:32000 --> 3200.00 s
window of 320.0 s, stride of 10.0 s
number of steps: 288


In [3]:
# Trajectory Smoothing: using a Savgol Filter in order to drop the noise due to the tracking procedure
smoothTrajs = get_smooth_trajs(rawTrajs, nDrops, 30, 2)

# MSD

In [ ]:
show_verb = False
run_windowed_analysis = False
animated_plot_verb = False
print(f"MSD Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")

%run ./analysis_modules/msd.ipynb

# VELOCITY DISTRIBUTION

In [4]:
show_verb = False
animated_plot_verb = False
print(f"Speed and Turning Angles Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_distr.ipynb

Speed and Turning Angles Analysis: show_verb = False, animated_plot_verb = False

 Global speed distribution analysis
Raw Trajectories
Blue Particles σ: 3.0812678386034857 ± 0.015939935826104385
Red Particle σ: 7.3063902778733985 ± 0.1325638679530156
Smooth Trajectories
Blue Particles σ: 2.6247331712013717 ± 0.016982879149678404
Red Particle σ: 7.501490328804058 ± 0.14385893343130896

 Windowed speed distribution Analysis


100%|██████████| 288/288 [01:04<00:00,  4.43it/s]


# TURNING ANGLES DISTRIBUTIONS

In [5]:
if 0:
    distribution = normal_distr
    distribution_str = "gaussian"
else:
    distribution = lorentian_distr
    distribution_str = "lorentian"

show_verb = False
animated_plot_verb = False
print(f"Speed and Turning Angles Analysis: show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/turning_angles.ipynb

Speed and Turning Angles Analysis: show_verb = False, animated_plot_verb = False

 Global turning angles analysis
Raw Trajectories
Blue Particles σ: 2.497021447528245 ± 0.4101053482508507, μ: 0.0055759244140203435 ± 0.43102324792811025
Red Particle σ: 1.8195193626205122 ± 0.17936967429817283, μ: 0.028399012774180964 ± 0.17941401010417252
Smooth Trajectories
Blue Particles σ: 0.3184052484952828 ± 0.0026005990592533417, μ: 0.0001197299050340255 ± 0.0025994981455607523
Red Particle σ: 0.14040069882588682 ± 0.0009333366055569999, μ: 0.0007676986138275656 ± 0.0009319299883564692

 Windowed turning angles analysis


100%|██████████| 288/288 [00:54<00:00,  5.31it/s]


# VELOCITY AUTOCORRELATION

In [ ]:
show_verb = False
run_windowed_analysis = False
overwrite = False
animated_plot_verb = False

print(f"Velocity Autocorrelation Analysis: show_verb = {show_verb}, run_windowed_analysis = {run_windowed_analysis}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/velocity_autocorrelation.ipynb

# RADIAL DISTRIBUTON FUNCTION

In [7]:
run_analysis_verb = True
show_verb = False
animated_plot_verb = True

print("Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}")
%run ./analysis_modules/radial_distribution_function.ipynb

Radial Distribution Function Analysis: run_analysis_verb = {run_analysis_verb}, show_verb = {show_verb}, animated_plot_verb = {animated_plot_verb}
RDF - Raw Trajectories
raw


KeyboardInterrupt: 

KeyboardInterrupt: 

# 3 BODY STRUCTURE METRIC

In [ ]:
run_analysis_verb = False
show_verb = False
animated_plot_verb = False

%run ./analysis_modules/3body_analysis.ipynb